In [1]:
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import heapq
# from sklearn.metrics import confusion_matrix
# # from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import precision_score
# from sklearn.metrics import average_precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import f1_score 
# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
# from skimage.filters import threshold_otsu
# similarity_case_1_p = '../../../results/RF/simliarity_measure/case_1_ref_mask_similarity_measure.tif'
# similarity_case_2_p = '../../../results/RF/simliarity_measure/case_2_ref_mask_similarity_measure.tif'
# similarity_case_3_p = '../../../results/RF/simliarity_measure/case_3_ref_mask_similarity_measure.tif'
# binary_change_map = '../../../results/RF/binary_change_D/change_map_case_1.tif'
# similarity_change_map = '../../../results/RF/simliarity_measure/optimal_threshold/sim-change_map_case_1.tif'
# source_proba_distr = '../../../results/RF/simliarity_measure/2018_rf_model_1.npy'
# target_proba_distri = '../../../results/RF/simliarity_measure/2019_rf_model_1.npy'

gt_source_path  = '../../../data/rasterized_samples/2018_rasterizedImage.tif'
gt_target_path = '../../../data/rasterized_samples/2019_rasterizedImage.tif'
pred_source_path = '../../../results/RF/2018_rf_case_2_map.tif'
pred_target_path = '../../../results/RF/2019_rf_case_3_map.tif'

In [ ]:
source_proba = np.load(source_proba_distr)

In [ ]:
source_proba = np.sort(source_proba, axis=1)[:, ::-1]

In [8]:
source_proba

array([[0.2154273 , 0.16797907, 0.12088084, ..., 0.0115641 , 0.00269574,
        0.00205739],
       [0.18054258, 0.17856504, 0.12668672, ..., 0.01568175, 0.00323386,
        0.00269574],
       [0.2257703 , 0.14244967, 0.08704204, ..., 0.00510253, 0.00360707,
        0.00301844],
       ...,
       [0.97664903, 0.01751358, 0.00188406, ..., 0.        , 0.        ,
        0.        ],
       [0.96421647, 0.02934249, 0.0022334 , ..., 0.        , 0.        ,
        0.        ],
       [0.95563428, 0.03953535, 0.00224455, ..., 0.        , 0.        ,
        0.        ]])

In [5]:
uncertainty = source_proba[:,0] - source_proba[:,1] 

In [7]:
uncertainty

array([0.04744823, 0.00197754, 0.08332063, ..., 0.95913544, 0.93487399,
       0.91609893])

In [2]:
gt_source_ = rasterio.open(gt_source_path).read(1).flatten().astype('int')
gt_target_ = rasterio.open(gt_target_path).read(1).flatten().astype('int')
pred_source_ = rasterio.open(pred_source_path).read(1).flatten()
pred_target_ = rasterio.open(pred_target_path).read(1).flatten()

In [3]:
gt_mask_nodata = (gt_source_ != 0) & (gt_target_ != 0)

In [4]:
gt_mask_nodata

array([False, False, False, ..., False, False, False])

In [5]:
def mask_and_extract(array, mask_array):
    array = np.ma.masked_array(array, mask=True)
    array.mask[mask_array] = False
    array_value = np.ma.compressed(array)
    return array_value
gt_source_ = mask_and_extract(gt_source_, gt_mask_nodata)
gt_target_ = mask_and_extract(gt_target_, gt_mask_nodata)
pred_source_ = mask_and_extract(pred_source_, gt_mask_nodata)
pred_target_ = mask_and_extract(pred_target_, gt_mask_nodata)

In [6]:
df = pd.DataFrame({'gt_source': gt_source_, 'gt_target': gt_target_, 'pred_source': pred_source_, 'pred_target': pred_target_})

In [7]:
dict_={1: 'Dense built-up area',
2: 'Diffuse built-up area',
 3: 'Industrial and commercial areas',
 4: 'Roads',
 5: 'Oilseeds (Rapeseed)',
 6: 'Straw cereals (Wheat, Triticale, Barley)',
 7: 'Protein crops (Beans / Peas)',
 8: 'Soy',
 9: 'Sunflower',
 10: 'Corn',
 12: 'Tubers/roots',
 13: 'Grasslands',
 14: 'Orchards and fruit growing',
 15: 'Vineyards',
 16: 'Hardwood forest',
 17: 'Softwood forest',
 18: 'Natural grasslands and pastures',
 19: 'Woody moorlands',
 23: 'Water'}

In [8]:
import time
start_time =time.time()
label = ['Dense built-up area', 'Diffuse built-up area', 'Industrial and commercial areas', 'Roads', 'Oilseeds (Rapeseed)', 'Straw cereals (Wheat, Triticale, Barley)', 'Protein crops (Beans / Peas)', 'Soy', 'Sunflower', 'Corn',  'Tubers/roots', 'Grasslands', 'Orchards and fruit growing', 'Vineyards', 'Hardwood forest', 'Softwood forest', 'Natural grasslands and pastures', 'Woody moorlands', 'Water']
df['gt_source'] = df['gt_source'].map(dict_)
df['gt_target'] = df['gt_target'].map(dict_)
df['pred_source'] = df['pred_source'].map(dict_)
df['pred_target'] = df['pred_target'].map(dict_)
print(" %s minute" % str((time.time() - start_time)/60))

 0.010872252782185872 minute


In [ ]:
import time
start_time =time.time()
df['gt_change']= df.apply(lambda x: str(x['gt_source']) + '-' + str(x['gt_target']), axis=1)
df['pred_change']= df.apply(lambda x: str(x['pred_source']) + '-' + str(x['pred_target']), axis=1)
print(" %s minutes" % str((time.time() - start_time)/60))

In [10]:
cm = pd.crosstab(df['gt_change'], df['pred_change'], rownames=['Ground Truth'], colnames=['Prediction'])

In [17]:
cm.to_csv('cm.csv')

In [7]:
# df['gt_change']= df.apply(lambda x: str(x['gt_source']) + '-' + str(x['gt_target']), axis=1)
# df['pred_change']= df.apply(lambda x: str(x['pred_source']) + '-' + str(x['pred_target']), axis=1)

In [ ]:
def changeErrorCheck(
    case,
    gt_source,
    gt_target,
    pred_source,
    pred_target,
    outdir
    ):
    # read rasters
    gt_source_ = rasterio.open(gt_source).read(1).flatten().astype('int')
    gt_target_ = rasterio.open(gt_target).read(1).flatten().astype('int')
    pred_source_ = rasterio.open(pred_source).read(1).flatten()
    pred_target_ = rasterio.open(pred_target).read(1).flatten()
    
    gt_mask_nodata = (gt_source_ != 0) & (gt_target_ != 0)
    
    def mask_and_extract(array, mask_array):
        array = np.ma.masked_array(array, mask=True)
        array.mask[mask_array] = False
        array_value = np.ma.compressed(array)
        return array_value

    gt_source_ = mask_and_extract(gt_source_, gt_mask_nodata)
    gt_target_ = mask_and_extract(gt_target_, gt_mask_nodata)
    pred_source_ = mask_and_extract(pred_source_, gt_mask_nodata)
    pred_target_ = mask_and_extract(pred_target_, gt_mask_nodata)
    
    # class label
    dict_={1: 'Dense built-up area',
    2: 'Diffuse built-up area',
     3: 'Industrial and commercial areas',
     4: 'Roads',
     5: 'Oilseeds (Rapeseed)',
     6: 'Straw cereals (Wheat, Triticale, Barley)',
     7: 'Protein crops (Beans / Peas)',
     8: 'Soy',
     9: 'Sunflower',
     10: 'Corn',
     12: 'Tubers/roots',
     13: 'Grasslands',
     14: 'Orchards and fruit growing',
     15: 'Vineyards',
     16: 'Hardwood forest',
     17: 'Softwood forest',
     18: 'Natural grasslands and pastures',
     19: 'Woody moorlands',
     23: 'Water'}
    
    df = pd.DataFrame({'gt_source': gt_source_, 'gt_target': gt_target_, 'pred_source': pred_source_, 'pred_target': pred_target_})
    # rename class to label
    df['gt_source'] = df['gt_source'].map(dict_)
    df['gt_target'] = df['gt_target'].map(dict_)
    df['pred_source'] = df['pred_source'].map(dict_)
    df['pred_target'] = df['pred_target'].map(dict_)
    
    
    df['gt_change']= df.apply(lambda x: str(x['gt_source']) + '-' + str(x['gt_target']), axis=1)
    df['pred_change']= df.apply(lambda x: str(x['pred_source']) + '-' + str(x['pred_target']), axis=1)
    
    cm = pd.crosstab(df['gt_change'], df['pred_change'], rownames=['Ground Truth'], colnames=['Prediction'])
    
    cm.to_csv("semantic_change_matrix" + case + '.csv')

if __name__ == '__main__':
    gt_source_path = '../../../data/rasterized_samples/2018_rasterizedImage.tif'
    gt_target_path = '../../../data/rasterized_samples/2019_rasterizedImage.tif'
    
    for case in ['1', '2', '3']:
        pred_source_path = '../../../results/RF/2018_rf_case_'+ case +'_map.tif'
        pred_target_path = '../../../results/RF/2019_rf_case_'+ case +'_map.tif'
        outdir_ = '../../../results/RF/binary_change_D'
        changeErrorCheck(case, gt_source_path, gt_target_path, pred_source_path, pred_target_path, outdir)